## Initial Setup

In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

In [ ]:
llm = os.getenv("LLM")
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")

llm = "openai/gpt-oss-20b"
api_key = ""
base_url = "https://api.groq.com/openai/v1"

model = ChatOpenAI(
    model = llm,
    api_key = api_key,
    base_url = base_url,
    streaming = False
)

## Query Enhancement Techniques

### Query Re-writing

Makes Query More specific and Detailed

In [4]:
query_rewrite_template = """
You are an AI assistant tasked with reformulating user queries to improve retrieval in a RAG system. 
Given the original query, rewrite it to be more specific, detailed, and likely to retrieve relevant information.
Original query: {original_query}
Rewritten query:
"""

query_rewrite_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=query_rewrite_template
)

In [5]:
def query_rewrite_generator(query:str) -> str:
    res = model.invoke(
        query_rewrite_prompt.invoke({"original_query":query})
    )
    return res.content

In [7]:
res = query_rewrite_generator("Can you tell me what is the solar system")

print(res)

Explain what constitutes the Solar System, detailing the Sun, the eight planets, dwarf planets, major moons, asteroid belt, Kuiper belt, Oort cloud, and its formation history.


### Step-Back Prompting

Generate More Broader and genralized Query

In [8]:
StepBack_prompting_template = """
You are an AI assistant tasked with generating broader, more general queries to improve context retrieval in a RAG system.
Given the original query, generate a step-back query that is more general and can help retrieve relevant background information.
Don't explain anything just answer the step-back query

Original query: {original_query}

Step-back query:
"""

StepBack_prompting_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=StepBack_prompting_template
)

In [9]:
def stepback_prompting_generator(query:str) -> str:
    res = model.invoke(
        StepBack_prompting_prompt.invoke({"original_query":query})
    )
    return res.content

In [11]:
res = stepback_prompting_generator("Can you tell me what is the solar system")

print(res)

What is a solar system?


### Sub-Query Decomposition

Decomposes the Queries into Different Sub-queries

In [15]:
subquery_decomposition_template = """
You are an AI assistant tasked with breaking down complex queries into simpler sub-queries for a RAG system.
Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query.

Original query: {original_query}

example: What are the impacts of climate change on the environment?

Sub-queries:
1. What are the impacts of climate change on biodiversity?
2. How does climate change affect the oceans?
3. What are the effects of climate change on agriculture?
4. What are the impacts of climate change on human health?
"""

subquery_decomposition_prompt = PromptTemplate(
    input_variables = ["original_query"],
    template = subquery_decomposition_template
)

In [16]:
def subquery_decomposition_generator(query:str) -> str:
    res = model.invoke(
        subquery_decomposition_prompt.invoke({"original_query":query})
    )
    return res.content

In [17]:
res = subquery_decomposition_generator("Can you tell me what is the solar system")
print(res)  

**Sub‑queries**

1. **What is the definition of the solar system and what are its primary components?**  
   (Clarify the concept, the Sun, planets, dwarf planets, moons, asteroid belt, Kuiper belt, Oort cloud, etc.)

2. **How are the major celestial bodies in the solar system structured and distributed?**  
   (Describe the orbital arrangement, relative sizes, distances, and types of bodies.)

3. **What is the formation and evolutionary history of the solar system?**  
   (Explain the nebular hypothesis, accretion, planetary differentiation, and current stage of development.)

4. **How do dynamic processes (orbital mechanics, gravitational interactions, resonance) govern the behavior and stability of the solar system?**  
   (Cover Keplerian orbits, perturbations, resonances, and long‑term stability.)


### HyDE (Hypothetical Document Embeddings)

<div align="center">
<img src="https://raw.githubusercontent.com/NirDiamant/RAG_Techniques/b9617dfe582aae0cd5333a964e6049a4133e04c4/images/HyDe.svg" height="600">
</div>

### HyPE (Hypothetical Prompt Embedding)

HyPE precomputes hypothetical questions during the indexing phase. And Retreival done usin Question-Quetion Matching 

In [23]:
hype_template = """
Analyze the input text and generate essential questions that, when answered,
capture the main points of the text. Each question should be one line, without numbering or prefixes.
Output formats: 
questions: list[str]

{chunk_text}

Questions:
"""

hype_prompt = PromptTemplate(
    input_variables = ["chunk_text"],
    template = hype_template
)

In [30]:
from pydantic import BaseModel

class Prompts(BaseModel):
    questions: list[str]

structured_model = model.with_structured_output(Prompts)
def hypothetical_prompt_generator(query:str) -> list[str]:
    res = structured_model.invoke(
        hype_prompt.invoke({"chunk_text":query})
    )
    return res.questions

In [31]:
from utils import get_pdf_chunks

In [32]:
res = get_pdf_chunks("2506.10943v2.pdf")

In [34]:
hypothetical_prompt_generator(res[10].page_content)

['How does Test-Time Training modify model weights during inference?',
 'What advantage does combining TTT with In-Context Learning provide in few-shot scenarios?',
 'How does SEAL implement TTT within its inner-loop optimization?',
 'Why is TTT considered more efficient than full-scale training in SEAL?',
 'In what way does the method trained on single-example TTT episodes generalize to the continued pretraining regime?',
 'What role does reinforcement learning play in improving LLM behavior, particularly in RLHF and verifiable reward settings?']

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def encode_pdf(chunks: list[str]):
    vector_store = None
    with ThreadPoolExecutor() as pool:
        futures = [pool.submit(hypothetical_prompt_generator, c) for c in chunks]

        for f in as_completed(futures), len(chunks):
            chunk, vectors = f.result()